In [1]:
%%capture
!pip install tensorboardX
!pip install efficientnet_pytorch
!pip gdown

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/Stanford997/CS599-Fake-Video-Detection.git

fatal: destination path 'CS599-Fake-Video-Detection' already exists and is not an empty directory.


In [4]:
cd CS599-Fake-Video-Detection

/content/CS599-Fake-Video-Detection


In [5]:
!git pull https://github.com/Stanford997/CS599-Fake-Video-Detection.git

From https://github.com/Stanford997/CS599-Fake-Video-Detection
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [6]:
import os
import shutil
import warnings

import albumentations as A
import numpy as np
import pandas as pd
import torch
import torch.multiprocessing
from torchvision.transforms import ToPILImage, ToTensor

from isplutils import utils, split

torch.multiprocessing.set_sharing_strategy('file_system')
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score
from tensorboardX import SummaryWriter
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import ImageChops, Image
from torchvision.models import InceptionOutputs

from architectures import fornet
from isplutils.data import FrameFaceIterableDataset, load_face

In [7]:
# Neural network type
# net_type = 'Xception'
# net_type = 'EfficientNetB4'
net_type = 'InceptionV3'
net_class = getattr(fornet, net_type)

In [8]:
# Initialization parameters
train_datasets = ['ff-c23-720-140-140']
val_datasets = ['ff-c23-720-140-140']
ffpp_df_path = '/content/drive/MyDrive/CS599/Dataset/image_df/df.pkl'
ffpp_faces_dir = '/content/drive/MyDrive/CS599/Dataset/image'
dfdc_df_path = None
dfdc_faces_dir = None
face_policy = 'scale'
face_size = 224
if net_type == 'InceptionV3':
    face_size = 299

batch_size = 100
initial_lr = 1e-5
validation_interval = 50
patience = 10
max_num_iterations = 3000
initial_model = None
train_from_scratch = False

max_train_samples = -1
max_val_samples = 6000

log_interval = 30
num_workers = 6
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
seed = 41

debug = False
suffix = None
enable_attention = True

weights_folder = f'weights/{net_type}'
logs_folder = f'logs/{net_type}'

In [9]:
def tb_attention(tb: SummaryWriter,
                 tag: str,
                 iteration: int,
                 net: nn.Module,
                 device: torch.device,
                 patch_size_load: int,
                 face_crop_scale: str,
                 val_transformer: A.BasicTransform,
                 root: str,
                 record: pd.Series,
                 ):
    # Crop face
    sample_t = load_face(record=record, root=root, size=patch_size_load, scale=face_crop_scale,
                         transformer=val_transformer)
    sample_t_clean = load_face(record=record, root=root, size=patch_size_load, scale=face_crop_scale,
                               transformer=ToTensorV2())
    if torch.cuda.is_available():
        sample_t = sample_t.cuda(device)
    # Transform
    # Feed to net
    with torch.no_grad():
        att: torch.Tensor = net.get_attention(sample_t.unsqueeze(0))[0].cpu()
    att_img: Image.Image = ToPILImage()(att)
    sample_img = ToPILImage()(sample_t_clean)
    att_img = att_img.resize(sample_img.size, resample=Image.NEAREST).convert('RGB')
    sample_att_img = ImageChops.multiply(sample_img, att_img)
    sample_att = ToTensor()(sample_att_img)
    tb.add_image(tag=tag, img_tensor=sample_att, global_step=iteration)


def batch_forward(net: nn.Module, device: torch.device, criterion, data: torch.Tensor, labels: torch.Tensor) -> (
        torch.Tensor, float, int):
    data = data.to(device)
    labels = labels.to(device)
    out = net(data)
    if isinstance(out, InceptionOutputs):
        out = out.logits
    pred = torch.sigmoid(out).detach().cpu().numpy()
    loss = criterion(out, labels)
    return loss, pred


def validation_routine(net, device, val_loader, criterion, tb, iteration, tag: str, loader_len_norm: int = None):
    net.eval()
    loader_len_norm = loader_len_norm if loader_len_norm is not None else val_loader.batch_size
    val_num = 0
    val_loss = 0.
    pred_list = list()
    labels_list = list()
    for val_data in tqdm(val_loader, desc='Validation', leave=False, total=len(val_loader) // loader_len_norm):
        batch_data, batch_labels = val_data

        val_batch_num = len(batch_labels)
        labels_list.append(batch_labels.flatten())
        with torch.no_grad():
            val_batch_loss, val_batch_pred = batch_forward(net, device, criterion, batch_data,
                                                           batch_labels)
        pred_list.append(val_batch_pred.flatten())
        val_num += val_batch_num
        val_loss += val_batch_loss.item() * val_batch_num

    # Logging
    val_loss /= val_num
    tb.add_scalar('{}/loss'.format(tag), val_loss, iteration)

    if isinstance(criterion, nn.BCEWithLogitsLoss):
        val_labels = np.concatenate(labels_list)
        val_pred = np.concatenate(pred_list)
        val_roc_auc = roc_auc_score(val_labels, val_pred)
        tb.add_scalar('{}/roc_auc'.format(tag), val_roc_auc, iteration)
        tb.add_pr_curve('{}/pr'.format(tag), val_labels, val_pred, iteration)

    return val_loss


def save_model(net: nn.Module, optimizer: optim.Optimizer,
               train_loss: float, val_loss: float,
               iteration: int, batch_size: int, epoch: int,
               path: str):
    path = str(path)
    state = dict(net=net.state_dict(),
                 opt=optimizer.state_dict(),
                 train_loss=train_loss,
                 val_loss=val_loss,
                 iteration=iteration,
                 batch_size=batch_size,
                 epoch=epoch)
    torch.save(state, path)

In [10]:
# Random initialization
np.random.seed(seed)
torch.random.manual_seed(seed)

# Load net
net: nn.Module = net_class().to(device)

# Loss and optimizers
criterion = nn.BCEWithLogitsLoss()

min_lr = initial_lr * 1e-5
optimizer = optim.Adam(net.get_trainable_parameters(), lr=initial_lr)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer,
    mode='min',
    factor=0.1,
    patience=patience,
    cooldown=2 * patience,
    min_lr=min_lr,
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
tag = utils.make_train_tag(net_class=net_class,
                           traindb=train_datasets,
                           face_policy=face_policy,
                           patch_size=face_size,
                           seed=seed,
                           suffix=suffix,
                           debug=debug,
                           )

# Model checkpoint paths
bestval_path = os.path.join(weights_folder, tag, 'bestval.pth')
last_path = os.path.join(weights_folder, tag, 'last.pth')
periodic_path = os.path.join(weights_folder, tag, 'it{:06d}.pth')

os.makedirs(os.path.join(weights_folder, tag), exist_ok=True)

# Load model
val_loss = min_val_loss = 10
epoch = iteration = 0
net_state = None
opt_state = None
if initial_model is not None:
    # If given load initial model
    print('Loading model form: {}'.format(initial_model))
    state = torch.load(initial_model, map_location='cpu')
    net_state = state['net']
elif not train_from_scratch and os.path.exists(last_path):
    print('Loading model form: {}'.format(last_path))
    state = torch.load(last_path, map_location='cpu')
    net_state = state['net']
    opt_state = state['opt']
    iteration = state['iteration'] + 1
    epoch = state['epoch']
if not train_from_scratch and os.path.exists(bestval_path):
    state = torch.load(bestval_path, map_location='cpu')
    min_val_loss = state['val_loss']
if net_state is not None:
    incomp_keys = net.load_state_dict(net_state, strict=False)
    print(incomp_keys)
if opt_state is not None:
    for param_group in opt_state['param_groups']:
        param_group['lr'] = initial_lr
    optimizer.load_state_dict(opt_state)

# Initialize Tensorboard
logdir = os.path.join(logs_folder, tag)
if iteration == 0:
    # If training from scratch or initialization remove history if exists
    shutil.rmtree(logdir, ignore_errors=True)

# TensorboardX instance
tb = SummaryWriter(logdir=logdir)
if iteration == 0:
    dummy = torch.randn((1, 3, face_size, face_size), device=device)
    dummy = dummy.to(device)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        tb.add_graph(net, [dummy, ], verbose=False)

transformer = utils.get_transformer(face_policy=face_policy, patch_size=face_size,
                                    net_normalizer=net.get_normalizer(), train=True)

# Datasets and data loaders
print('Loading data')
# Check if paths for DFDC and FF++ extracted faces and DataFrames are provided
for dataset in train_datasets:
    if dataset.split('-')[0] == 'dfdc' and (dfdc_df_path is None or dfdc_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for DFDC faces for training!')
    elif dataset.split('-')[0] == 'ff' and (ffpp_df_path is None or ffpp_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for FF++ faces for training!')
for dataset in val_datasets:
    if dataset.split('-')[0] == 'dfdc' and (dfdc_df_path is None or dfdc_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for DFDC faces for validation!')
    elif dataset.split('-')[0] == 'ff' and (ffpp_df_path is None or ffpp_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for FF++ faces for validation!')
# Load splits with the make_splits function
splits = split.make_splits(dfdc_df=dfdc_df_path, ffpp_df=ffpp_df_path, dfdc_dir=dfdc_faces_dir, ffpp_dir=ffpp_faces_dir,
                           dbs={'train': train_datasets, 'val': val_datasets})
train_dfs = [splits['train'][db][0] for db in splits['train']]
train_roots = [splits['train'][db][1] for db in splits['train']]
val_roots = [splits['val'][db][1] for db in splits['val']]
val_dfs = [splits['val'][db][0] for db in splits['val']]

Parameters
{'face': 'scale',
 'net': 'InceptionV3',
 'seed': 41,
 'size': 299,
 'traindb': 'ff-c23-720-140-140'}
Tag: net-InceptionV3_traindb-ff-c23-720-140-140_face-scale_size-299_seed-41


/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1692: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


Loading data


In [12]:
train_dataset = FrameFaceIterableDataset(roots=train_roots,
                                         dfs=train_dfs,
                                         scale=face_policy,
                                         num_samples=max_train_samples,
                                         transformer=transformer,
                                         size=face_size,
                                         )

val_dataset = FrameFaceIterableDataset(roots=val_roots,
                                       dfs=val_dfs,
                                       scale=face_policy,
                                       num_samples=max_val_samples,
                                       transformer=transformer,
                                       size=face_size,
                                       )

train_loader = DataLoader(train_dataset, num_workers=num_workers, batch_size=batch_size, )
val_loader = DataLoader(val_dataset, num_workers=num_workers, batch_size=batch_size, )

print('Training samples: {}'.format(len(train_dataset)))
print('Validation samples: {}'.format(len(val_dataset)))

Training samples: 51528
Validation samples: 6000


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [13]:
stop = False
while not stop:

    # Training
    optimizer.zero_grad()

    train_loss = train_num = 0
    train_pred_list = []
    train_labels_list = []
    for train_batch in tqdm(train_loader, desc='Epoch {:03d}'.format(epoch), leave=False,
                            total=len(train_loader) // train_loader.batch_size):
        net.train()
        batch_data, batch_labels = train_batch

        train_batch_num = len(batch_labels)
        train_num += train_batch_num
        train_labels_list.append(batch_labels.numpy().flatten())

        train_batch_loss, train_batch_pred = batch_forward(net, device, criterion, batch_data, batch_labels)
        train_pred_list.append(train_batch_pred.flatten())

        if torch.isnan(train_batch_loss):
            raise ValueError('NaN loss')

        train_loss += train_batch_loss.item() * train_batch_num

        # Optimization
        train_batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Logging
        if iteration > 0 and (iteration % log_interval == 0):
            train_loss /= train_num
            tb.add_scalar('train/loss', train_loss, iteration)
            tb.add_scalar('lr', optimizer.param_groups[0]['lr'], iteration)
            tb.add_scalar('epoch', epoch, iteration)

            # Checkpoint
            save_model(net, optimizer, train_loss, val_loss, iteration, batch_size, epoch, last_path)
            train_loss = train_num = 0

        # Validation
        if iteration > 0 and (iteration % validation_interval == 0):

            # Model checkpoint
            save_model(net, optimizer, train_loss, val_loss, iteration, batch_size, epoch,
                       periodic_path.format(iteration))

            # Train cumulative stats
            train_labels = np.concatenate(train_labels_list)
            train_pred = np.concatenate(train_pred_list)
            train_labels_list = []
            train_pred_list = []

            train_roc_auc = roc_auc_score(train_labels, train_pred)
            tb.add_scalar('train/roc_auc', train_roc_auc, iteration)
            tb.add_pr_curve('train/pr', train_labels, train_pred, iteration)

            # Validation
            val_loss = validation_routine(net, device, val_loader, criterion, tb, iteration, 'val')
            tb.flush()

            # LR Scheduler
            lr_scheduler.step(val_loss)

            # Model checkpoint
            if val_loss < min_val_loss:
                min_val_loss = val_loss
                save_model(net, optimizer, train_loss, val_loss, iteration, batch_size, epoch, bestval_path)

            # Attention
            if enable_attention and hasattr(net, 'get_attention'):
                net.eval()
                # For each dataframe show the attention for a real,fake couple of frames
                for df, root, sample_idx, tag in [
                    (train_dfs[0], train_roots[0], train_dfs[0][train_dfs[0]['label'] == False].index[0],
                     'train/att/real'),
                    (train_dfs[0], train_roots[0], train_dfs[0][train_dfs[0]['label'] == True].index[0],
                     'train/att/fake'),
                ]:
                    record = df.loc[sample_idx]
                    tb_attention(tb, tag, iteration, net, device, face_size, face_policy,
                                 transformer, root, record)

            if optimizer.param_groups[0]['lr'] == min_lr:
                print('Reached minimum learning rate. Stopping.')
                stop = True
                break

        iteration += 1

        if iteration > max_num_iterations:
            print('Maximum number of iterations reached')
            stop = True
            break

        # End of iteration

    epoch += 1

# Needed to flush out last events
tb.close()

print('Completed')

Epoch 000:   0%|          | 0/3 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 000: 64it [04:08,  3.51s/it]
Validation: 0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this wil

Maximum number of iterations reached


Completed


In [14]:
from google.colab import drive
import shutil

# upload the log and model to google drive
drive.mount('/content/drive')

source_folder = "/content/CS599-Fake-Video-Detection/logs"
target_folder = "/content/drive/MyDrive/CS599/runs"

shutil.copytree(source_folder, target_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/CS599/runs'

In [15]:
source_folder = "/content/CS599-Fake-Video-Detection/weights"
target_folder = "/content/drive/MyDrive/CS599/weights"

shutil.copytree(source_folder, target_folder)

'/content/drive/MyDrive/CS599/weights'